# Data Anaysis

## load data

In [1]:
from data_pipeline.open_data_marine_weather.core import OpenDataMarineWeather
from data_pipeline.open_data_raw_material_price.core import OpenDataRawMaterialPrice
from data_pipeline.open_data_terrestrial_weather.core import OpenDataTerrestrialWeather

open_data_terrestrial_weather = OpenDataTerrestrialWeather(
    date="201908"
)
open_data_marine_weather = OpenDataMarineWeather(
    date="201908"
)
open_data_raw_material_price = OpenDataRawMaterialPrice(
    date="201908"
)

t_df = open_data_terrestrial_weather.load()
w_df = open_data_marine_weather.load()    
o_df= open_data_raw_material_price.load()

2020-03-31 19:30:15:manager:<INFO> 1 files is loaded from public_data/open_data_terrestrial_weather/origin/csv/2014-2020.csv in s3 'production-bobsim'
2020-03-31 19:30:16:manager:<INFO> 1 files is loaded from public_data/open_data_marine_weather/origin/csv/2014-2020.csv in s3 'production-bobsim'
2020-03-31 19:30:19:manager:<INFO> 1 files is loaded from public_data/open_data_raw_material_price/origin/csv/201908.csv in s3 'production-bobsim'
2020-03-31 19:30:22:manager:<INFO> 1 files is loaded from public_data/open_data_terrestrial_weather/origin/csv/2014-2020.csv in s3 'production-bobsim'
2020-03-31 19:30:23:manager:<INFO> 1 files is loaded from public_data/open_data_marine_weather/origin/csv/2014-2020.csv in s3 'production-bobsim'
2020-03-31 19:30:26:manager:<INFO> 1 files is loaded from public_data/open_data_raw_material_price/origin/csv/201908.csv in s3 'production-bobsim'


## weather data

### terrestrial weather

In [2]:
t_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214325 entries, 0 to 214324
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   일시            214325 non-null  datetime64[ns]
 1   평균기온(°C)      214086 non-null  float32       
 2   최저기온(°C)      214300 non-null  float32       
 3   최고기온(°C)      214305 non-null  float16       
 4   강수 계속시간(hr)   22288 non-null   float16       
 5   일강수량(mm)      77862 non-null   float16       
 6   최대 풍속(m/s)    214217 non-null  float16       
 7   평균 풍속(m/s)    214143 non-null  float16       
 8   최소 상대습도(pct)  214246 non-null  float16       
 9   평균 상대습도(pct)  213919 non-null  float32       
 10  합계 일조시간(hr)   213966 non-null  float16       
dtypes: datetime64[ns](1), float16(7), float32(3)
memory usage: 6.9 MB


In [3]:
t_df.isnull().sum()/214325*100

일시               0.000000
평균기온(°C)         0.111513
최저기온(°C)         0.011665
최고기온(°C)         0.009332
강수 계속시간(hr)     89.600840
일강수량(mm)        63.671060
최대 풍속(m/s)       0.050391
평균 풍속(m/s)       0.084918
최소 상대습도(pct)     0.036860
평균 상대습도(pct)     0.189432
합계 일조시간(hr)      0.167503
dtype: float64

In [28]:
t_dropped_df = t_df.drop('합계 일조시간(hr)', axis= 1)

In [39]:
import pandas as pd

def fillna_with_mean(df: pd.DataFrame):
    return df.interpolate(method='linear', limit_direction='both')

def fill_nan(x):
    columns_with_zero = ['강수 계속시간(hr)' , '일강수량(mm)']
    columns_with_mean = [
    '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', 
    '최대 풍속(m/s)', '평균 풍속(m/s)', '최소 상대습도(pct)', '평균 상대습도(pct)'
    ]
    if x.name in columns_with_mean :
        return x.interpolate(method='linear', limit_direction='both')
    elif x.name in columns_with_zero:
        return x.fillna(0)
    else:
        return x

In [40]:
t_not_null_df = t_dropped_df.apply(fill_nan, axis=0)
t_not_null_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214325 entries, 0 to 214324
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   일시            214325 non-null  datetime64[ns]
 1   평균기온(°C)      214325 non-null  float32       
 2   최저기온(°C)      214325 non-null  float32       
 3   최고기온(°C)      214325 non-null  float16       
 4   강수 계속시간(hr)   214325 non-null  float16       
 5   일강수량(mm)      214325 non-null  float16       
 6   최대 풍속(m/s)    214325 non-null  float16       
 7   평균 풍속(m/s)    214325 non-null  float16       
 8   최소 상대습도(pct)  214325 non-null  float16       
 9   평균 상대습도(pct)  214325 non-null  float32       
dtypes: datetime64[ns](1), float16(6), float32(3)
memory usage: 6.5 MB
